In [1]:
import glob
from PIL import Image
import os
import cv2
import numpy as np
from keras.models import Sequential,load_model
from keras.layers import Dense,Conv2D,MaxPooling2D,Dropout,Flatten,Activation,BatchNormalization
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
save_file_directory='/content/drive/My Drive/Colab Notebooks/해커톤/data/'
poketmon=['꼬부기','이상해씨','파이리','버터플','뮤','푸린','마자용','나옹','디그다','야돈','고라파덕','꼬마돌','쥬레곤','뽀뽀라','또가스','고오스','잉어킹','메타몽','이브이','피카츄','잠만보','롱스톤','치코리타','브케인','리아코','토게피','데덴네','마릴','나몰빼미']
attr=[]
label=[]

def one_hot_encoder(string):
    poketmon=['꼬부기','이상해씨','파이리','버터플','뮤','푸린','마자용','나옹','디그다','야돈','고라파덕','꼬마돌','쥬레곤','뽀뽀라','또가스','고오스','잉어킹','메타몽','이브이','피카츄','잠만보','롱스톤','치코리타','브케인','리아코','토게피','데덴네','마릴','나몰빼미']
    result=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    index = poketmon.index(string)
    result[index]+=1
    return np.array(result)

for name in poketmon:
    for index in range(1,21):
        filename = save_file_directory+name+'/'+name+"_"+str(index)+'.jpg'
        img= load_img(filename)
        x=img_to_array(img)
        attr.append(x)
        label.append(one_hot_encoder(name))

attr = np.array(attr)
label = np.array(label)

In [12]:
MODEL_DIR = '/content/drive/My Drive/Colab Notebooks/해커톤/data/models/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)
model_path = '/content/drive/My Drive/Colab Notebooks/해커톤/data/models/{epoch:02d} - {val_loss:2f}.hdf5'
checkpointer = ModelCheckpoint(filepath=model_path, monitor='val_loss', verbose=1,save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss',patience=10)
    
model=Sequential()
model.add(Conv2D(32,kernel_size=(3,3),input_shape=(150,150,3),activation='relu'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32,(3,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(64,(3,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Activation('relu'))
model.add(Dense(29,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
history = model.fit(attr,label,validation_data=(attr,label),epochs=100,batch_size=145, verbose=1,callbacks=[early_stopping_callback,checkpointer])



Epoch 1/100
4/4 [==============================] - ETA: 0s - loss: 5.7001 - accuracy: 0.0517
Epoch 00001: val_loss improved from inf to 9.68732, saving model to /content/drive/My Drive/Colab Notebooks/해커톤/data/models/01 - 9.69.hdf5
4/4 [==============================] - 1s 188ms/step - loss: 5.7001 - accuracy: 0.0517 - val_loss: 9.6873 - val_accuracy: 0.0328
Epoch 2/100
4/4 [==============================] - ETA: 0s - loss: 3.3295 - accuracy: 0.0724
Epoch 00002: val_loss improved from 9.68732 to 6.41878, saving model to /content/drive/My Drive/Colab Notebooks/해커톤/data/models/02 - 6.42.hdf5
4/4 [==============================] - 1s 149ms/step - loss: 3.3295 - accuracy: 0.0724 - val_loss: 6.4188 - val_accuracy: 0.0345
Epoch 3/100
4/4 [==============================] - ETA: 0s - loss: 3.2528 - accuracy: 0.1069
Epoch 00003: val_loss improved from 6.41878 to 4.60830, saving model to /content/drive/My Drive/Colab Notebooks/해커톤/data/models/03 - 4.61.hdf5
4/4 [=====================

In [32]:
def output(path):
    width = 150
    height = 150
    img_jpg = Image.open(path)
    resize_img = img_jpg.resize((width, height), Image.ANTIALIAS)
    input_data=img_to_array(resize_img)
    model = load_model('/content/drive/My Drive/Colab Notebooks/해커톤/data/models/final_model.hdf5')
    print(model.predict_classes(np.array([input_data])))
    return poketmon[model.predict_classes(np.array([input_data]))[0]]
    

In [21]:
output('/content/drive/My Drive/Colab Notebooks/해커톤/시연/시연_1.jpg')

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
[2]


array([2])

In [22]:
output('/content/drive/My Drive/Colab Notebooks/해커톤/시연/시연_2.jpg')

[6]


array([6])

In [23]:
output('/content/drive/My Drive/Colab Notebooks/해커톤/시연/시연_3.jpg')

[21]


array([21])

In [33]:
output('/content/drive/My Drive/Colab Notebooks/해커톤/data/이상해씨/이상해씨_12.jpg')

[1]


'이상해씨'

In [25]:
output('/content/drive/My Drive/Colab Notebooks/해커톤/data/브케인/브케인_12.jpg')

[23]


array([23])

In [26]:
output('/content/drive/My Drive/Colab Notebooks/해커톤/data/파이리/파이리_12.jpg')

[2]


array([2])

In [29]:
output('/content/drive/My Drive/Colab Notebooks/해커톤/시연/test.jpg')

[8]


array([8])

In [30]:
output('/test2.jpg')

[14]


array([14])

# 데이터 불리기

In [ ]:
# from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

# save_file_directory='C:/Users/Administrator/Dropbox/2020해커톤/data/'
# poketmon=['꼬부기','이상해씨','파이리','버터플','뮤','푸린','마자용','나옹','디그다','야돈','고라파덕','꼬마돌','쥬레곤','뽀뽀라','또가스','고오스','잉어킹','메타몽','이브이','피카츄','잠만보','롱스톤','치코리타','브케인','리아코','토게피','데덴네','마릴','나몰빼미','라프라스']
# datagen = ImageDataGenerator(
#         rotation_range=40,
#         width_shift_range=0.2,
#         height_shift_range=0.2,
#         shear_range=0.2,
#         zoom_range=0.2,
#         horizontal_flip=True,
#         fill_mode='nearest')

# for name in poketmon:
#     for index in range(1,21):
#         i=0
#         filename = name+'_'+str(index)+'.jpg'
#         img = load_img(save_file_directory+name+'/'+filename)  # PIL 이미지
#         x = img_to_array(img)  # (3, 150, 150) 크기의 NumPy 배열
#         x = x.reshape((1,) + x.shape)  # (1, 3, 150, 150) 크기의 NumPy 배열
#         for batch in datagen.flow(x, batch_size=1,save_to_dir=save_file_directory+name+'/', save_prefix='꼬부기'+'_'+str(index), save_format='jpeg'):
#             i += 1
#             if i > 20:
#                 break
# # 아래 .flow() 함수는 임의 변환된 이미지를 배치 단위로 생성해서
# # 지정된 `preview/` 폴더에 저장합니다.


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/Administrator/Dropbox/2020해커톤/data/꼬부기/꼬부기_2.jpg'